In [1]:
import numpy as np
from recstate2mode import UW_,UV_,C_,mu_,Sigma_,G_matrix

In [2]:
gamma = np.array([0,0],dtype = np.complex128)
phi = np.array([0,0.4],dtype = np.complex128)
zeta = np.array([0.2,0.3],dtype = np.complex128)
theta = 0.2
psi = 0
theta1 = 0.3
psi1 = 0.5
cutoff_dim = 4
cutoff= 4
cutoff_plus = 10


In [3]:
old_state = np.random.rand(cutoff_dim,cutoff_dim) + 1.0j*np.random.rand(cutoff_dim,cutoff_dim)
old_state  /= np.linalg.norm(old_state)

In [5]:
W = UW_(phi,theta1,psi1)
V = UV_(theta,psi)
    
C = C_(gamma, W, zeta, V)
mu = mu_(gamma, W, zeta, V)
Sigma = Sigma_(W, zeta, V)

cutoff = old_state.shape[0]
dtype = old_state.dtype


sqrt = np.sqrt(np.arange(cutoff+1, dtype=dtype))

G_00pq = np.zeros((cutoff, cutoff), dtype=dtype)


G_00pq[0,0] = C
for q in range(1, cutoff):
    G_00pq[0,q] = (mu[3]*G_00pq[0,q-1] - Sigma[3,3]*sqrt[q-1]*G_00pq[0,q-2])/sqrt[q]


for p in range(1,cutoff):
    for q in range(0,cutoff):
        G_00pq[p,q] = (mu[2]*G_00pq[p-1,q] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,q] - Sigma[2,3]*sqrt[q]*G_00pq[p-1,q-1])/sqrt[p]

In [6]:
G = G_matrix(gamma, phi, theta1, psi1, zeta, theta, psi,cutoff)

In [15]:
# R_00^jk = a_dagger^j \G_00pq> b^k  * |old_state>

R = np.zeros((cutoff, cutoff, cutoff+1, cutoff+1), dtype=dtype)

a2 = np.zeros((cutoff,cutoff), dtype=dtype)
for i in range(cutoff-1):
    a2[i+1,i] = np.sqrt(i+1)
b = np.zeros((cutoff,cutoff), dtype=dtype)
for i in range(cutoff-1):
    b[i,i+1] = np.sqrt(i+1)
      
    
for j in range(cutoff):
    for k in range(cutoff):
            G_00pq2 = G_00pq
            for _ in range(j):
                G_00pq2 = a2@G_00pq2
            for _ in range(k):
                G_00pq2 = G_00pq2@b
            R[0,0,j,k] = np.sum(G_00pq2*old_state)
    
#R_0n^jk
for n in range(1,cutoff):
    for k in range(0,cutoff): 
        for j in range(0,cutoff):
         
            R[0,n,j,k] = mu[1]/sqrt[n]*R[0,n-1,j,k] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[0,n-2,j,k] - Sigma[1,2]/sqrt[n]*R[0,n-1,j+1,k] - Sigma[1,3]/sqrt[n]*R[0,n-1,j,k+1]


for m in range(1,cutoff):
    for n in range(0,cutoff):
        for j in range(0,cutoff-m):
            for k in range(0,cutoff-m-j):
                R[m,n,j,k] = mu[0]/sqrt[m]*R[m-1,n,j,k] - Sigma[0,0]/sqrt[m]*sqrt[m-1]*R[m-2,n,j,k] - Sigma[0,1]*sqrt[n]/sqrt[m]*R[m-1,n-1,j,k] - Sigma[0,2]/sqrt[m]*R[m-1,n,j+1,k] - Sigma[0,3]/sqrt[m]*R[m-1,n,j,k+1]


In [16]:
#Result from recursive process
R[:,:,0,0]

array([[ 0.19972895+3.71409373e-01j,  0.03702599+3.88707907e-01j,
        -0.16343969+3.03213756e-01j, -0.16590057+1.90543339e-01j],
       [ 0.15876648+2.69824581e-02j, -0.05446824+2.42633452e-01j,
        -0.00463241+2.27415768e-01j, -0.07147674+1.92134980e-01j],
       [ 0.05291939+7.92691860e-02j,  0.02444894+1.12988970e-01j,
        -0.05162278-1.08639394e-01j,  0.06343097-3.95620386e-02j],
       [-0.08801777+1.26480542e-01j,  0.02835399-2.19066757e-04j,
        -0.01829033-1.88024136e-01j,  0.04317521-1.30923140e-01j]])

In [17]:
#Result from G matrix
np.einsum("cd,abcd->ab",old_state,G)

array([[ 0.19972895+3.71409373e-01j,  0.03702599+3.88707907e-01j,
        -0.16343969+3.03213756e-01j, -0.16590057+1.90543339e-01j],
       [ 0.15876648+2.69824581e-02j, -0.05446824+2.42633452e-01j,
        -0.00463241+2.27415768e-01j, -0.07147674+1.92134980e-01j],
       [ 0.05291939+7.92691860e-02j,  0.02444894+1.12988970e-01j,
        -0.05162278-1.08639394e-01j,  0.06343097-3.95620386e-02j],
       [-0.08801777+1.26480542e-01j,  0.02835399-2.19066757e-04j,
        -0.01829033-1.88024136e-01j,  0.04317521-1.30923140e-01j]])